# Recommendations with IBM

## Introduction

This project analyzes user interactions with articles on the IBM Watson Studio platform to build recommendation systems. The goal is to recommend articles to users based on their previous interactions and the behavior of similar users.

## Table of Contents

1. [Exploratory Data Analysis](#eda)
2. [Rank Based Recommendations](#rank)
3. [User-User Based Collaborative Filtering](#user-user)
4. [Content Based Recommendations](#content-based)
5. [Matrix Factorization](#matrix-fact)
6. [Evaluation](#evaluation)

## Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import re

%matplotlib inline

# Load data
df = pd.read_csv('data/user-item-interactions.csv')
df_content = pd.read_csv('data/articles_community.csv')
del df['Unnamed: 0']
del df_content['Unnamed: 0']

# Show df to understand the data
df.head()

<a id='eda'></a>
## Part I: Exploratory Data Analysis

Before building recommendation systems, let's explore the data to understand:
- The distribution of how many articles a user interacts with
- The number of unique articles that have an interaction
- The number of unique users in the dataset
- The most viewed articles

In [ ]:
# Explore the data
print(f"Shape of df: {df.shape}")
print(f"\nFirst few rows:")
df.head(10)

In [ ]:
# Check article content data
df_content.head()

### Question 1: What is the distribution of how many articles a user interacts with in the dataset?

In [ ]:
# Distribution of user interactions
user_interactions = df.groupby('email')['article_id'].count()
user_interactions.describe()

In [ ]:
# Visualize the distribution
plt.figure(figsize=(12, 6))
plt.hist(user_interactions, bins=50)
plt.xlabel('Number of Article Interactions')
plt.ylabel('Number of Users')
plt.title('Distribution of User-Article Interactions')
plt.show()

# Calculate required statistics
median_val = user_interactions.median()
max_views_by_user = user_interactions.max()

print(f"Median number of interactions per user: {median_val}")
print(f"Maximum number of interactions by a user: {max_views_by_user}")

### Question 2: Explore the number of unique articles and users

In [ ]:
# Calculate key statistics
unique_articles = df['article_id'].nunique()
total_articles = df.shape[0]  # Total interactions
unique_users = df['email'].nunique()
user_article_interactions = df.shape[0]

print(f"Number of unique articles: {unique_articles}")
print(f"Number of user-article interactions: {user_article_interactions}")
print(f"Number of unique users: {unique_users}")

### Question 3: Which article has the most interactions?

In [ ]:
# Find the most viewed article
article_views = df.groupby('article_id')['email'].count().sort_values(ascending=False)
most_viewed_article_id = article_views.index[0]
max_views = article_views.iloc[0]

print(f"Most viewed article ID: {most_viewed_article_id}")
print(f"Number of views: {max_views}")
print(f"\nTop 10 most viewed articles:")
print(article_views.head(10))

### Create User-ID Mapping

We'll create a mapping from email addresses to user IDs for easier processing.

In [ ]:
# Create a user_id column by mapping email to a unique numeric ID
def email_mapper(df=df):
    """
    Maps email addresses to user IDs
    
    Args:
        df: pandas dataframe with email column
    
    Returns:
        email_encoded: list of user_ids corresponding to emails in df
    """
    coded_dict = {
        email: num 
        for num, email in enumerate(df['email'].unique(), start=1)
    }
    return [coded_dict[val] for val in df['email']]

df['user_id'] = email_mapper(df)
del df['email']

# Show the updated dataframe
df.head()

<a id='rank'></a>
## Part II: Rank Based Recommendations

For new users or cold-start scenarios, we can recommend the most popular articles. This approach recommends articles based on the number of interactions they have received.

In [ ]:
def get_top_articles(n, df=df):
    """
    Returns the top n article titles with the most interactions
    
    Args:
        n: int, number of top articles to return
        df: pandas dataframe of user-item interactions
    
    Returns:
        top_articles: list of top n article titles
    """
    # Get article counts
    article_counts = df.groupby('title')['user_id'].count().sort_values(ascending=False)
    top_articles = list(article_counts.index[:n])
    
    return top_articles

def get_top_article_ids(n, df=df):
    """
    Returns the top n article IDs with the most interactions
    
    Args:
        n: int, number of top articles to return
        df: pandas dataframe of user-item interactions
    
    Returns:
        top_articles: list of top n article IDs as strings
    """
    # Get article counts by article_id
    article_counts = df.groupby('article_id')['user_id'].count().sort_values(ascending=False)
    top_articles = list(article_counts.index[:n].astype(str))
    
    return top_articles

# Test the functions
print("Top 10 Article Titles:")
top_10_titles = get_top_articles(10)
for i, title in enumerate(top_10_titles, 1):
    print(f"{i}. {title}")

print("\nTop 10 Article IDs:")
top_10_ids = get_top_article_ids(10)
print(top_10_ids)

<a id='user-user'></a>
## Part III: User-User Based Collaborative Filtering

In this section, we build a collaborative filtering system. We'll find similar users based on their article interactions and recommend articles that similar users have interacted with.

### Create User-Item Matrix

In [ ]:
def create_user_item_matrix(df, fill_value=0):
    """
    Creates a user-item matrix where 1 indicates an interaction
    
    Args:
        df: pandas dataframe of user-item interactions
        fill_value: value to fill for non-interactions (default 0)
    
    Returns:
        user_item: pandas dataframe with users as rows and articles as columns
    """
    # Create a user-item matrix with 1's for interactions
    # fill_value is used by unstack() before binary conversion
    user_item = df.groupby(['user_id', 'article_id'])['title'].count().unstack(fill_value=fill_value)
    
    # Convert to binary (1 for interaction, 0 for no interaction)
    # Note: After this, all values become 0 or 1 regardless of fill_value
    user_item = (user_item > 0).astype(int)
    
    return user_item

# Create the user-item matrix
user_item = create_user_item_matrix(df)

print(f"User-Item Matrix Shape: {user_item.shape}")
print(f"Number of users: {user_item.shape[0]}")
print(f"Number of articles: {user_item.shape[1]}")
user_item.head()

### Find Similar Users

In [ ]:
def find_similar_users(user_id, user_item=user_item, include_similarity=False):
    """
    Finds users similar to a given user based on article interactions
    
    Args:
        user_id: int, the user_id to find similar users for
        user_item: pandas dataframe, user-item matrix
        include_similarity: bool, whether to include similarity scores in output
    
    Returns:
        similar_users: numpy array of similar user_ids sorted by similarity,
                      or list of lists [[user_id, similarity], ...] if include_similarity=True
    """
    # Check if user exists
    if user_id not in user_item.index:
        if include_similarity:
            return []
        return np.array([])
    
    # Calculate similarity using dot product (number of common articles)
    user_vector = user_item.loc[user_id].values.reshape(1, -1)
    similarities = user_item.dot(user_vector.T).flatten()
    
    # Sort by similarity (excluding the user itself)
    similar_indices = np.argsort(similarities)[::-1]
    similar_users = user_item.index[similar_indices].values
    similar_scores = similarities[similar_indices]
    
    # Remove the user itself
    mask = similar_users != user_id
    similar_users = similar_users[mask]
    similar_scores = similar_scores[mask]
    
    if include_similarity:
        return [[int(user), float(score)] for user, score in zip(similar_users, similar_scores)]
    
    return similar_users

# Test the function
test_user = 1
similar = find_similar_users(test_user)
print(f"Most similar users to user {test_user}: {similar[:10]}")

### Helper Functions for Recommendations

In [ ]:
def get_article_names(article_ids, df=df):
    """
    Get article names from article IDs
    
    Args:
        article_ids: list of article_ids
        df: pandas dataframe of user-item interactions
    
    Returns:
        article_names: list of article titles
    """
    article_names = []
    
    for article_id in article_ids:
        # Get the title for this article_id
        title = df[df['article_id'] == float(article_id)]['title'].iloc[0]
        article_names.append(title)
    
    return article_names

def get_user_articles(user_id, user_item=user_item, df=df):
    """
    Get articles that a user has interacted with
    
    Args:
        user_id: int, the user_id
        user_item: pandas dataframe, user-item matrix
        df: pandas dataframe of user-item interactions
    
    Returns:
        article_ids: list of article_ids the user has interacted with
        article_names: list of article titles corresponding to article_ids
    """
    # Get articles where user has interacted (value = 1)
    if user_id not in user_item.index:
        return [], []
    
    user_row = user_item.loc[user_id]
    article_ids = list(user_row[user_row == 1].index.astype(float))
    
    # Get article names
    article_names = get_article_names(article_ids, df)
    
    # Convert article_ids to strings for consistency
    article_ids = [str(int(aid)) for aid in article_ids]
    
    return article_ids, article_names

In [ ]:
def get_top_sorted_users(user_id, df=df, user_item=user_item):
    """
    Get users sorted by similarity to the given user
    
    Args:
        user_id: int, the user_id
        df: pandas dataframe of user-item interactions
        user_item: pandas dataframe, user-item matrix
    
    Returns:
        neighbors_df: pandas dataframe of similar users with similarity scores
    """
    # Find similar users with similarity scores
    similar_users_with_sim = find_similar_users(user_id, user_item, include_similarity=True)
    
    # Check if user exists
    if not similar_users_with_sim:
        return pd.DataFrame()
    
    # Extract user ids and similarities
    user_ids = [u[0] for u in similar_users_with_sim]
    similarities = [u[1] for u in similar_users_with_sim]
    
    # Calculate number of interactions for each user
    num_interactions = [user_item.loc[uid].sum() for uid in user_ids]
    
    # Create dataframe
    neighbors_df = pd.DataFrame({
        'neighbor_id': user_ids,
        'similarity': similarities,
        'num_interactions': num_interactions
    })
    
    # Sort by similarity (descending) then by num_interactions (descending)
    neighbors_df = neighbors_df.sort_values(['similarity', 'num_interactions'], ascending=False)
    
    return neighbors_df

### User-User Recommendation Functions

In [ ]:
def user_user_recs(user_id, m=10):
    """
    Recommend articles to a user based on similar users' interactions
    
    Args:
        user_id: int, the user_id to make recommendations for
        m: int, number of recommendations to return
    
    Returns:
        recs: list of article_ids to recommend
    """
    # Get articles the user has already seen
    user_articles_ids, user_articles_names = get_user_articles(user_id, user_item, df)
    user_articles = set(user_articles_ids)
    
    # Find similar users
    similar_users = find_similar_users(user_id, user_item)
    
    # Get articles from similar users
    rec_articles = []
    article_counts = {}
    
    for sim_user in similar_users:
        sim_user_articles_ids, _ = get_user_articles(sim_user, user_item, df)
        
        for article_id in sim_user_articles_ids:
            if article_id not in user_articles:
                article_counts[article_id] = article_counts.get(article_id, 0) + 1
    
    # Sort by popularity among similar users
    sorted_articles = sorted(article_counts.items(), key=lambda x: x[1], reverse=True)
    recs = [article_id for article_id, count in sorted_articles[:m]]
    
    # If we don't have enough recommendations, add popular articles
    if len(recs) < m:
        top_articles = get_top_article_ids(m * 2, df)
        for article_id in top_articles:
            if article_id not in user_articles and article_id not in recs:
                recs.append(article_id)
                if len(recs) >= m:
                    break
    
    return recs[:m]

# Test the function
test_recs = user_user_recs(1, m=10)
print(f"Recommendations for user 1: {test_recs}")

In [ ]:
def get_ranked_article_unique_counts(article_ids, user_item=user_item):
    """
    Get articles ranked by the number of unique users who interacted with them
    
    Args:
        article_ids: list of article_ids to rank
        user_item: pandas dataframe, user-item matrix
    
    Returns:
        ranked_article_unique_counts: list of tuples (article_id, unique_user_count)
                                      sorted by unique_user_count in descending order
    """
    # Count unique users for each article in the provided list
    article_counts = []
    
    for article_id in article_ids:
        # Convert to float for consistency with user_item columns
        article_id_float = float(article_id)
        
        # Check if article exists in user_item matrix
        if article_id_float in user_item.columns:
            # Count users who interacted with this article (value = 1)
            count = user_item[article_id_float].sum()
            article_counts.append([int(article_id_float), int(count)])
    
    # Sort by count (descending)
    ranked_article_unique_counts = sorted(article_counts, key=lambda x: x[1], reverse=True)
    
    return ranked_article_unique_counts

# Test
test_articles = [1320, 232, 844]
ranked_articles = get_ranked_article_unique_counts(test_articles, user_item)
print(f"Ranked articles {test_articles}:")
print(ranked_articles)

In [ ]:
def user_user_recs_part2(user_id, m=10):
    """
    Improved version of user_user_recs with better handling of edge cases
    
    Args:
        user_id: int, the user_id to make recommendations for
        m: int, number of recommendations to return
    
    Returns:
        recs: list of article_ids to recommend
        rec_names: list of article names corresponding to recs
    """
    # For new users, return most popular articles
    if user_id not in user_item.index:
        recs = get_top_article_ids(m, df)
        rec_names = get_article_names(recs, df)
        return recs, rec_names
    
    # Use the original user_user_recs function
    recs = user_user_recs(user_id, m)
    rec_names = get_article_names(recs, df)
    
    return recs, rec_names

# Test
test_recs, test_names = user_user_recs_part2(1, m=5)
print("Recommendations for user 1:")
for i, (rec_id, name) in enumerate(zip(test_recs, test_names), 1):
    print(f"{i}. {name} (ID: {rec_id})")

### Calculate Required Variables for Testing

In [ ]:
# Calculate required variables for grading

# Most similar user to user 1
similar_to_1 = find_similar_users(1, user_item)
user1_most_sim = similar_to_1[0] if len(similar_to_1) > 0 else None

# 6th most similar user to user 2
similar_to_2 = find_similar_users(2, user_item)
user2_6th_sim = similar_to_2[5] if len(similar_to_2) > 5 else None

# 10th most similar user to user 131
similar_to_131 = find_similar_users(131, user_item)
user131_10th_sim = similar_to_131[9] if len(similar_to_131) > 9 else None

# Recommendations for a new user (user not in dataset)
new_user_id = max(user_item.index) + 1
new_user_recs = get_top_article_ids(10, df)

print(f"Most similar user to user 1: {user1_most_sim}")
print(f"6th most similar user to user 2: {user2_6th_sim}")
print(f"10th most similar user to user 131: {user131_10th_sim}")
print(f"\nRecommendations for new user: {new_user_recs}")

<a id='content-based'></a>
## Part IV: Content Based Recommendations

In this section, we use article content (text) to make recommendations. We'll use TF-IDF to vectorize article text, apply dimensionality reduction with LSA, and cluster similar articles using K-Means.

### Prepare Article Content Data

In [ ]:
# Check article content data
print("Article content columns:", df_content.columns.tolist())
print(f"\nShape: {df_content.shape}")
df_content.head()

In [ ]:
# Clean and prepare text data
def clean_text(text):
    """
    Clean text for processing
    
    Args:
        text: string
    
    Returns:
        cleaned text string
    """
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

# Combine title and doc_body for content
df_content['content'] = df_content['doc_full_name'].fillna('') + ' ' + df_content['doc_description'].fillna('')
df_content['content'] = df_content['content'].apply(clean_text)

# Remove rows with empty content
df_content = df_content[df_content['content'].str.len() > 0]

print(f"Articles with content: {len(df_content)}")
df_content[['article_id', 'doc_full_name', 'content']].head()

### TF-IDF Vectorization and LSA

In [ ]:
# Create TF-IDF matrix
# Parameters:
#   - max_features=1000: Limit vocabulary to top 1000 terms to reduce dimensionality
#   - stop_words='english': Remove common English words that don't add meaning
#   - max_df=0.8: Ignore terms that appear in more than 80% of documents
#   - min_df=2: Ignore terms that appear in fewer than 2 documents
tfidf = TfidfVectorizer(max_features=1000, stop_words='english', max_df=0.8, min_df=2)
tfidf_matrix = tfidf.fit_transform(df_content['content'])

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Apply LSA (Latent Semantic Analysis) for dimensionality reduction
# Use min() to ensure n_components doesn't exceed matrix dimensions
n_components = min(100, tfidf_matrix.shape[0] - 1, tfidf_matrix.shape[1] - 1)
lsa = TruncatedSVD(n_components=n_components, random_state=42)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

print(f"LSA matrix shape: {lsa_matrix.shape}")
print(f"Explained variance ratio: {lsa.explained_variance_ratio_.sum():.4f}")

### K-Means Clustering

In [ ]:
# Apply K-Means clustering
# n_clusters=20: Group articles into 20 thematic clusters
# random_state=42: Ensure reproducibility
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
df_content['cluster'] = kmeans.fit_predict(lsa_matrix)

print(f"Number of articles per cluster:")
print(df_content['cluster'].value_counts().sort_index())

### Content-Based Recommendation Functions

In [ ]:
def get_similar_articles(article_id, df=df):
    """
    Find similar articles based on title similarity (using clusters)
    
    Args:
        article_id: float or int, the article_id
        df: pandas dataframe of user-item interactions
    
    Returns:
        similar_articles: list of similar article_ids as strings
    """
    # Check if article exists
    article_id = float(article_id)
    
    # Get the article's title
    article_titles = df[df['article_id'] == article_id]['title']
    if len(article_titles) == 0:
        return []
    
    article_title = article_titles.iloc[0]
    
    # Find all articles with the same title (title cluster)
    same_title_articles = df[df['title'] == article_title]['article_id'].unique()
    
    # Remove the input article_id from the list
    similar_articles = [str(int(aid)) for aid in same_title_articles if aid != article_id]
    
    return similar_articles

# Test the function
test_article_id = df['article_id'].iloc[0]
similar = get_similar_articles(test_article_id, df)
print(f"Articles similar to {test_article_id}:")
print(similar)

In [ ]:
def make_content_recs(article_id, n, df=df):
    """
    Make content-based recommendations for an article
    Returns similar articles ranked by popularity
    
    Args:
        article_id: int or float, the article_id
        n: int, number of recommendations to return
        df: pandas dataframe of user-item interactions
    
    Returns:
        n_ranked_similar_articles: list of n similar article_ids ranked by popularity
        n_ranked_article_names: list of article names corresponding to article_ids
    """
    # Get similar articles (articles in same title cluster)
    similar_article_ids = get_similar_articles(article_id, df)
    
    if not similar_article_ids:
        # If no similar articles, return popular articles
        top_ids = get_top_article_ids(n, df)
        top_names = get_article_names([float(aid) for aid in top_ids], df)
        return top_ids, top_names
    
    # Convert to int for get_ranked_article_unique_counts
    # Note: similar_article_ids are strings, convert to int
    similar_article_ids_int = [int(aid) for aid in similar_article_ids]
    
    # Rank by unique user counts
    ranked_counts = get_ranked_article_unique_counts(similar_article_ids_int, user_item)
    
    # Extract article_ids and limit to n
    n_ranked_similar_articles = [str(aid) for aid, count in ranked_counts[:n]]
    
    # Get article names
    n_ranked_article_names = get_article_names([float(aid) for aid in n_ranked_similar_articles], df)
    
    return n_ranked_similar_articles, n_ranked_article_names

# Test
test_article_id = df['article_id'].iloc[0]
content_recs_ids, content_recs_names = make_content_recs(test_article_id, n=5, df=df)
print(f"Content-based recommendations for article {test_article_id}:")
for i, (aid, name) in enumerate(zip(content_recs_ids, content_recs_names), 1):
    print(f"{i}. {name} (ID: {aid})")

<a id='matrix-fact'></a>
## Part V: Matrix Factorization (SVD)

Finally, we use Singular Value Decomposition (SVD) to factorize the user-item matrix and make recommendations based on latent features.

In [ ]:
# Perform SVD on the user-item matrix
from scipy.sparse.linalg import svds

# Convert to numpy array for SVD
user_item_matrix = user_item.values

# Choose number of latent factors
k = 50

# Perform SVD
U, sigma, Vt = svds(user_item_matrix, k=k)

# Convert sigma to diagonal matrix
sigma = np.diag(sigma)

print(f"U shape: {U.shape}")
print(f"Sigma shape: {sigma.shape}")
print(f"Vt shape: {Vt.shape}")

# Reconstruct the matrix
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(predicted_ratings, 
                                    index=user_item.index, 
                                    columns=user_item.columns)

print(f"\nPredicted ratings shape: {predicted_ratings_df.shape}")

In [ ]:
def get_svd_similar_article_ids(article_id, vt, user_item=user_item, include_similarity=False):
    """
    Find similar articles using SVD-based latent features
    
    Args:
        article_id: float or int, the article_id
        vt: numpy array, Vt matrix from SVD decomposition
        user_item: pandas dataframe, user-item matrix
        include_similarity: bool, whether to include similarity scores
    
    Returns:
        similar_articles: list of similar article_ids as strings,
                         or list of lists [[article_id, similarity], ...] if include_similarity=True
    """
    article_id = float(article_id)
    
    # Check if article exists in the matrix
    if article_id not in user_item.columns:
        if include_similarity:
            return []
        return []
    
    # Get the article's latent feature vector from Vt
    article_idx = list(user_item.columns).index(article_id)
    article_vector = vt[:, article_idx].reshape(1, -1)
    
    # Calculate similarity with all articles
    # vt should be transposed to get shape (n_articles, k)
    similarities = cosine_similarity(article_vector, vt.T).flatten()
    
    # Get indices of most similar articles (excluding the article itself)
    similar_indices = np.argsort(similarities)[::-1]
    
    # Filter out the article itself
    similar_indices = [idx for idx in similar_indices if user_item.columns[idx] != article_id]
    
    if include_similarity:
        # Return list of [article_id, similarity]
        result = []
        for idx in similar_indices:
            col_val = user_item.columns[idx]
            # Convert to string, handling both float and int types
            article_id_str = str(int(float(col_val)))
            result.append([article_id_str, float(similarities[idx])])
        return result
    else:
        # Get article_ids as strings
        similar_articles = []
        for idx in similar_indices:
            col_val = user_item.columns[idx]
            # Convert to string, handling both float and int types
            article_id_str = str(int(float(col_val)))
            similar_articles.append(article_id_str)
        return similar_articles

# Test the function
test_article = user_item.columns[0]
svd_similar = get_svd_similar_article_ids(test_article, Vt, user_item)[:5]
print(f"SVD-based articles similar to {test_article}:")
print(svd_similar)

In [ ]:
def svd_recommendations(user_id, m=10):
    """
    Make recommendations using SVD-predicted ratings
    
    Args:
        user_id: int, the user_id
        m: int, number of recommendations to return
    
    Returns:
        recs: list of recommended article_ids
    """
    if user_id not in predicted_ratings_df.index:
        # Return popular articles for new users
        return get_top_article_ids(m, df)
    
    # Get user's predicted ratings
    user_predictions = predicted_ratings_df.loc[user_id]
    
    # Get articles user hasn't interacted with
    user_articles_ids, _ = get_user_articles(user_id, user_item, df)
    user_articles = set(user_articles_ids)
    
    # Sort by predicted rating
    sorted_predictions = user_predictions.sort_values(ascending=False)
    
    # Filter out articles user has already seen
    recs = []
    for article_id in sorted_predictions.index:
        article_id_str = str(article_id)
        if article_id_str not in user_articles:
            recs.append(article_id_str)
            if len(recs) >= m:
                break
    
    return recs

# Test
svd_recs = svd_recommendations(1, m=5)
print(f"SVD-based recommendations for user 1: {svd_recs}")

<a id='evaluation'></a>
## Part VI: Evaluation and Conclusion

### Comparison of Recommendation Methods

We have implemented four different recommendation approaches:

1. **Rank-Based Recommendations**: Simple approach that recommends the most popular articles. Works well for new users (cold start) but doesn't personalize.

2. **User-User Collaborative Filtering**: Finds similar users based on interaction patterns and recommends articles that similar users liked. Works well when we have sufficient user interaction data.

3. **Content-Based Recommendations**: Uses article content (text) to find similar articles. Good for recommending articles similar to what a user has already read, even for articles with few interactions.

4. **Matrix Factorization (SVD)**: Discovers latent features in user-item interactions to make predictions. Can capture complex patterns and make personalized recommendations.

### Key Findings

- The dataset is sparse, with many users having only a few interactions
- Popular articles have significantly more interactions than average
- Different recommendation methods have different strengths for different scenarios

### Recommendations for Deployment

A hybrid approach would work best:
- Use rank-based for completely new users
- Use collaborative filtering for users with sufficient history
- Use content-based to discover similar articles
- Use SVD for personalized predictions when data is available

In [ ]:
# Summary statistics
print("=" * 60)
print("RECOMMENDATION SYSTEM SUMMARY")
print("=" * 60)
print(f"\nDataset Statistics:")
print(f"  - Total interactions: {user_article_interactions}")
print(f"  - Unique users: {unique_users}")
print(f"  - Unique articles: {unique_articles}")
print(f"  - Median interactions per user: {median_val}")
print(f"  - Max interactions by a user: {max_views_by_user}")
print(f"  - Most viewed article ID: {most_viewed_article_id}")
print(f"  - Views of most popular article: {max_views}")

print(f"\nRecommendation Models Implemented:")
print(f"  1. Rank-Based Recommendations")
print(f"  2. User-User Collaborative Filtering")
print(f"  3. Content-Based Recommendations (TF-IDF + LSA + KMeans)")
print(f"  4. Matrix Factorization (SVD)")

print(f"\nKey Variables for Testing:")
print(f"  - user1_most_sim: {user1_most_sim}")
print(f"  - user2_6th_sim: {user2_6th_sim}")
print(f"  - user131_10th_sim: {user131_10th_sim}")
print(f"  - new_user_recs (first 5): {new_user_recs[:5]}")
print("=" * 60)

## Next Steps

To further improve the recommendation system:

1. **Implement A/B testing** to compare different recommendation strategies in production
2. **Add temporal features** to account for article freshness and user behavior over time
3. **Incorporate implicit feedback** like reading time, clicks, and scrolling behavior
4. **Build a hybrid model** that combines multiple approaches with learned weights
5. **Add diversity** to recommendations to avoid filter bubbles
6. **Implement online learning** to update recommendations in real-time
7. **Add evaluation metrics** like precision@k, recall@k, and NDCG